Natural Observer

A project to collect the thousands of observations of the natural world from Reddit (and maybe eventually other social media). Photos, identification, and any location information are collated to create a usable dataset for citizen science networks such as eBird and iNaturalist. We hope...

Created by:
Lindsey Parkinson, Thomas Oliver, and Roman Grisch 

In [8]:
import praw
import json
import pandas as pd
import numpy as np
from collections import defaultdict
import re
import datetime

#redditkeys.json contains all the information necessary to use the Reddit API

with open("/home/lindsey/Documents/Git_Projects/Biodiversity_Scraping/redditkeys.json") as infile:
    credentials = json.load(infile)
reddit = praw.Reddit(client_id = credentials["client_id"],
                     client_secret = credentials["client_secret"],
                     user_agent=credentials["user_agent"],
                     username=credentials["username"],
                     password=credentials["password"])

print(reddit.user.me())

Parker09


In [47]:
date_list = []
author_list = []
id_list = []
link_flair_text_list = []
#num_comments_list = []
#score_list = []
title_list = []
#upvote_ratio_list = []
url_list = []
top_comment_list = []


#subreddits we want to scrape information from
subreddit_list=  ['whatisthisfish',
                  'whatsthisfish'
                  #'whatsthisbug',
                  #'whatsthisbird',
                  #'whatsthissnake',
                 ]

#What information we want from each subreddit post
for subred in subreddit_list:
    subreddit = reddit.subreddit(subred)
    top_post = subreddit.top(limit = 100)  
    
    for sub in top_post: 
        #if link_flair_text_list.append(sub.link_flair_text != 'Solved'):
            #continue
        
        date_list.append(datetime.datetime.fromtimestamp(sub.created_utc))
        author_list.append(sub.author)
        id_list.append(sub.id)        
        link_flair_text_list.append(sub.link_flair_text)
        #num_comments_list.append(sub.num_comments)
        #score_list.append(sub.score)
        title_list.append(sub.title)
        #upvote_ratio_list.append(sub.upvote_ratio) 
        url_list.append(sub.url)
        
        #for top_level_comment in sub.comments:
         #   top_comment_list.append(top_level_comment.body)
        
    print(subred, 'completed; ', end='')
    print('total', len(author_list), 'posts scraped')
    


whatisthisfish completed; total 100 posts scraped
whatsthisfish completed; total 200 posts scraped


In [48]:
def convert(row, col = "URL"):
    """
    This function will convert strings into hyperlinks, makes it easier to pull images
    """
    return "<a href='{}'>{}</a>".format(row[col], row.name)

In [49]:
df = pd.DataFrame({'Date': date_list,
                   'ID':id_list, 
                   'Author':author_list, 
                   'Title':title_list,
                   #'Count_of_Comments':num_comments_list,
                   #'Upvote_Count':score_list,
                   #'Upvote_Ratio':upvote_ratio_list,
                   'Flair':link_flair_text_list,
                   'URL':url_list
                  })



In [50]:
df['URL'] = df.apply(convert, axis = 1)

In [51]:
df

,Date,ID,Author,Title,Flair,URL
0,2020-02-17 19:45:23,f5denb,officialjoeyacnl,found at a pet store any ideas? image is a kno...,Solved,<a href='https://i.redd.it/f6ntf9ys4jh41.jpg'>...
1,2020-07-27 00:07:36,hyfvpv,xItsMeGradyMC,"What's this fish? Oh that, that's a bluegill",Solved,<a href='https://i.redd.it/k6rb9y0wy9d51.png'>...
2,2020-04-07 04:46:15,fwcjf9,Shinobi-butterstick,what type of fish is this?,Solved,<a href='https://i.redd.it/yxwvpygg7br41.jpg'>...
3,2020-07-29 23:44:45,i09fvp,PhotonicBoom21,FAQ: Common sunfish in North America,Moderator News,<a href='https://i.redd.it/brfh2rkh9vd51.jpg'>...
4,2020-02-17 14:48:48,f58z6m,jcole315315,"Caught in New Jersey, what is this fish?",Unsolved,<a href='https://i.redd.it/ljqoyz43ohh41.jpg'>...
...,...,...,...,...,...,...
195,2020-08-03 01:37:53,i2maf5,patriot-renegade,"My dad caught this by accident in Puget Sound,...","Identified, high confidence",<a href='https://i.redd.it/if6r0xredoe51.jpg'>...
196,2020-07-09 18:51:49,ho6lgj,_Atheist_Jesus_,Found in brackish water near Assateague Island...,"Identified, high confidence",<a href='https://i.redd.it/xb9mwq633v951.jpg'>...
197,2020-06-05 10:04:59,gx0egd,TheSailorSquid,Does anyone know what this crazy colourful fis...,"Identified, high confidence",<a href='https://i.redd.it/pp3dsnp3u1351.jpg'>...
198,2020-04-23 04:51:25,g6eycu,blueflower999,"What’s this fish? Found in Victoria, Australia...",Possible ID(s) suggested,<a href='https://i.redd.it/graieaexehu41.jpg'>...


In [72]:
comments = defaultdict(list)
# for ID in id_list:
#     submission = reddit.submission(str(ID))
#     for top_level_comment in submission.comments:
#         comments[submission.title].append(top_level_comment.body)
#             #print(top_level_comment[0])
            
for ID in id_list:
    submission = reddit.submission(str(ID))
    for top_level_comment in submission.comments:
        if submission_comms is not None:
            comments[submission.title].append(top_level_comment.body)
        else:
            comments[submission.title].append("NA")

In [70]:
print(len(comments))
print(comments)

defaultdict(<class 'list'>, {'Fish kept in a Hibachi restaurant fish tank (non-edible?) what fish is this?': ['Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian arowana - Scleropages sp', 'Asian

In [64]:
top_comment = list()
#for x in comments.values():
#if x in comments.values(): 
#    top_comment.append(x[0])
#else:
#    top_comment.apend("NA")
    
    
for key, val in comments.items():
    if val is not None:
        top_comment.append(val[0])
    else:
        top_comment.apend("NA")
        

In [65]:
top_comment


['Flowerhorn',
 'I wonder if it calls me pink lung.',
 "Scarus coeruleus, blue parrotfish! They are indeed that blue!  \n\n\nThe diets of these fish are comprised of algae- they use their powerful beaks to snap off chunks of stony coral. The algae living in these chunks of coral are eaten, but the fish can't do anything with the coral's skeleton, thus excreting it as sand (yes, out of the rear!). That's where a lot of Hawaii's white-sand beaches come from... the waste of the parrotfish!",
 'Recently we have been seeing a *lot* of sunfish on this subreddit, so this post is meant to help aid some beginners. Here is a [helpful guide](https://fishingbooker.com/blog/types-of-sunfish-in-north-america) with bit more detail than the info-graphic.\n\nIf anyone has helpful tips for identifying the many types of sunfish, feel free to chime in!',
 'Seaweed blenny.',
 'Looks like a type of Brittle Starfish with super long legs.',
 "It's an egg casing for a whelk or other species of large sea snail.

In [66]:
df["Top Comment"] = top_comment

ValueError: Length of values (199) does not match length of index (200)

In [ ]:
df

In [23]:
#Import and download NLP tools

import nltk
#nltk.download('punkt')
#nltk.download('maxent_ne_chunker')
#nltk.download('words')
#nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /home/lindsey/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /home/lindsey/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/lindsey/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [24]:
practice = df["Title"].copy
print(practice)

print(df["Title"].to_list())

<bound method NDFrame.copy of 0    found at a pet store any ideas? image is a kno...
1         What's this fish? Oh that, that's a bluegill
2                           what type of fish is this?
3                 FAQ: Common sunfish in North America
4             Caught in New Jersey, what is this fish?
5    Anybody know what species this is? Found in Ja...
6    Caught in key west, Florida. Never seen anythi...
7    I found this video and the fish is hella cute ...
8    What's this crustacean? Has a short lobster li...
9    Moray eel caught off the coast of Singapore. A...
Name: Title, dtype: object>
['found at a pet store any ideas? image is a known image too', "What's this fish? Oh that, that's a bluegill", 'what type of fish is this?', 'FAQ: Common sunfish in North America', 'Caught in New Jersey, what is this fish?', 'Anybody know what species this is? Found in Jacksonville, FL swimming under a leaf. Inshore.', 'Caught in key west, Florida. Never seen anything like it.', 'I found t

In [25]:
#A function to pull location information from Reddit post titles

#with open('sample.txt', 'r') as f:
out_str = ","
sample = out_str.join(df['Title'])


sentences = nltk.sent_tokenize(sample)
tokenized_sentences = [nltk.word_tokenize(sentence) for sentence in sentences]
tagged_sentences = [nltk.pos_tag(sentence) for sentence in tokenized_sentences]
chunked_sentences = nltk.ne_chunk_sents(tagged_sentences, binary=True)

def extract_entity_names(t):
    entity_names = []

    if hasattr(t, 'label') and t.label:
        if t.label() == 'NE':
            entity_names.append(' '.join([child[0] for child in t]))
        else:
            for child in t:
                entity_names.extend(extract_entity_names(child))

    return entity_names

entity_names = []
for tree in chunked_sentences:
    # Print results per sentence
    # print extract_entity_names(tree)

    entity_names.extend(extract_entity_names(tree))

# Print all entity names
#print entity_names

# Print unique entity names
print(entity_names)
    

['FAQ', 'North America', 'Caught', 'New Jersey', 'Anybody', 'Jacksonville', 'Caught', 'Florida', 'Florida', 'Moray', 'Singapore']


In [ ]:
#The function grabs all the location words but is also attracted to the word "caught"